<p align="center">
<img src="https://www.uao.edu.co/wp-content/uploads/2024/12/uao-logo-2-04.webp" width=15%>


<h2>UNIVERSIDAD AUTÓNOMA DE OCCIDENTE</strong></h2>
<h3>03/06/2025 CALI - COLOMBIA</strong></h3>
<h3><strong>MAESTRIA EN INTELIGENCIA ARTIFICIAL Y CIENCIA DE DATOS</strong></h3>
<h3><strong>ETL (EXTRACT, TRANSFORM AND LOAD)</strong></h3>
<h3><strong>EJERCICIO EN CLASE 2 </strong> TRANSFORMACIONES</h3>
<h3><strong>Profesor:</strong> JAVIER ALEJANDRO VERGARA ZORRILLA</h3>
<h3><strong>Alumno:</strong>><font color='lighblue'> 22500214 Yoniliman Galvis Aguirre </font></h3>

# EJERCICIO ETL #2

## Contexto
Este conjunto de datos proviene de una empresa de comercio electrónico, que contiene diversa información sobre los productos disponibles en la tienda.

## Ejercicio
*   Realiza todas las transformaciones necesarias para obtener un conjunto de datos limpio con las características requeridas para entrenar un modelo de aprendizaje automático que prediga si un producto es nuevo o usado.

*   Una característica es una columna con información importante o relevante para resolver el problema.

*   Toma en cuenta todas las consideraciones y supuestos que necesites. En la carpeta de Google Drive puedes encontrar el archivo data_clean.csv, el cual puedes usar como ejemplo de salida.

*   Realiza todo el Análisis Exploratorio de Datos (EDA) que consideres necesario, utiliza gráficos como apoyo y aplica todas las transformaciones requeridas.

# Verificar Kernel
Verificamos si el ambiente jupyter esta ejecutando el kernel en el entorno correcto, el resultado de las dos rutas debe coincidir, de lo contrario se debe de cambiar el kernel del jupyter notebook, una opcion es correr el enviroment desde poetry, en la terminal ejecute:
```bash
poetry run jupyter notebook
```
esto abrirá una version web de jupyter, en otro caso cambie el kernel y use los venv disponibles

Si el notebook esta ejecutando un kernel diferente a la carpeta del proyecto cuando instale librerías se presentarán fallas en la ejecucion del código del notebook

In [1]:
import subprocess
import shutil

# Ejecutar el comando de poetry desde Python
result = subprocess.run(['poetry', 'env', 'info', '--path'], capture_output=True, text=True)

# Obtener la ruta del entorno virtual
env_path = result.stdout.strip()

# Obtener la ruta del ejecutable de Python activo
python_path = shutil.which("python")

print(f"El entorno virtual activo de poetry está en: {env_path}")
print(f"El entorno virtual activo del kernel en el notebook está en: {python_path}")

El entorno virtual activo de poetry está en: /home/ygalvis/Documents/Study/ETL_Ejercicio2/.venv
El entorno virtual activo del kernel en el notebook está en: /home/ygalvis/Documents/Study/ETL_Ejercicio2/.venv/bin/python


# Cargar librerías

In [2]:
import pandas as pd
import jsonlines
from tqdm import tqdm

# Cargar Datos desde archivo JSONL

In [4]:
# Esta funcion va a Leer el archivo JSONL usando la librería jsonlines

def leer_jsonl(ruta_archivo, ruta_exportar):
    """
    *   Vamos a leer un archivo JSONL
    *   Eliminamos las columnas con valores nulos ó listas vacías ([])
    *   Exportamos los encabezados y la primera fila a un archivo de texto para identificar columnas diccionario

    Args:
        ruta_archivo (str): La ruta al archivo JSONL.
        ruta_exportar (str): La ruta al archivo de texto donde se exportarán los encabezados y la primera fila.

    Returns:
        pd.DataFrame: Un DataFrame de pandas con las columnas vacías eliminadas.
    """
    data = []
    try:
        with jsonlines.open(ruta_archivo) as reader:
            for obj in tqdm(reader, desc="Leyendo líneas del archivo JSONL"):
                data.append(obj)
    except FileNotFoundError:
        print("Tenemos problemas en encontrar el archivo, Verifica y corrige la ruta.")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error
    except jsonlines.InvalidLineError as e:
        print(f"Tenemos problemas en leer una línea del archivo, verifica el json o intenta otro método: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error

    # Convertir la lista de diccionarios a un DataFrame de pandas
    df = pd.DataFrame(data)

    # Eliminar columnas que contienen solo valores nulos o listas vacías ([])
    df = df.dropna(axis=1, how='all')
    df = df[[col for col in df if not all(isinstance(i, list) and len(i) == 0 for i in df[col])]]

    # Exportar los encabezados de las columnas y la primera fila a un archivo de texto
    with open(ruta_exportar, "w") as f:
        columnas = df.columns.tolist()
        valores = df.iloc[0].astype(str).tolist()
        
        max_len = max(len(col) for col in columnas)  # Longitud máxima de columna

        for col, val in zip(columnas, valores):
            f.write("{:>{width}} : {}\n".format(col, val, width=max_len))  # Alinear nombres

    return df

In [5]:
# traer datos del dataset usando la funcion
ruta_archivo = 'Dataset/MLA_100k.jsonlines'
ruta_exportar = 'Dataset/header_firstrow.txt'
df = leer_jsonl(ruta_archivo, ruta_exportar)

# Muestra todas las columnas pero en formato vertical y presenta la primera fila
print(df.head(1).T)
pd.options.display.max_rows = None  

Leyendo líneas del archivo JSONL: 100000it [00:04, 21097.75it/s]


                                                                                  0
seller_address                    {'comment': '', 'longitude': -58.3986709, 'id'...
warranty                                                                       None
sub_status                                                                       []
condition                                                                       new
seller_contact                                                                 None
deal_ids                                                                         []
base_price                                                                     80.0
shipping                          {'local_pick_up': True, 'methods': [], 'tags':...
non_mercado_pago_payment_methods  [{'description': 'Transferencia bancaria', 'id...
seller_id                                                                  74952096
variations                                                                  

                                                                                  0
seller_address                    {'comment': '', 'longitude': -58.3986709, 'id'...
warranty                                                                       None
sub_status                                                                       []
condition                                                                       new
seller_contact                                                                 None
deal_ids                                                                         []
base_price                                                                     80.0
shipping                          {'local_pick_up': True, 'methods': [], 'tags':...
non_mercado_pago_payment_methods  [{'description': 'Transferencia bancaria', 'id...
seller_id                                                                  74952096
variations                                                                  

In [11]:
# Función para identificar columnas con diccionarios o listas
columnas_anidadas = []

for col in df.columns:
    # Ignorar columnas vacías
    if df[col].dropna().empty:
        continue
    
    # Verificar si el primer valor no nulo es un diccionario o lista
    if isinstance(df[col].dropna().iloc[0], (dict, list)):
        columnas_anidadas.append(col)

print("Columnas con estructuras anidadas:", columnas_anidadas)

Columnas con estructuras anidadas: ['seller_address', 'sub_status', 'seller_contact', 'deal_ids', 'shipping', 'non_mercado_pago_payment_methods', 'variations', 'location', 'attributes', 'tags', 'descriptions', 'pictures', 'geolocation']


In [7]:
# Columnas con datos anidados que queremos expandir
columnas_a_expandir = columnas_anidadas

for col in columnas_a_expandir:
    if col in df.columns:
        df_expanded = pd.json_normalize(df[col], sep="_")

        # Evitar columnas duplicadas eliminando claves ya presentes en df
        df_expanded = df_expanded.drop(columns=[c for c in df_expanded.columns if c in df.columns], errors='ignore')

        # Eliminar la columna original y unir las nuevas columnas
        df = df.drop(columns=[col]).join(df_expanded)

# Mostrar el DataFrame procesado
print(df.head())

             warranty condition  base_price  seller_id site_id  \
0                None       new        80.0   74952096     MLA   
1  NUESTRA REPUTACION      used      2650.0   42093335     MLA   
2                None      used        60.0  133384258     MLA   
3                None       new       580.0  143001605     MLA   
4      MI REPUTACION.      used        30.0   96873449     MLA   

  listing_type_id   price buying_mode listing_source parent_item_id  ...  \
0          bronze    80.0  buy_it_now                  MLA568261029  ...   
1          silver  2650.0  buy_it_now                  MLA561574487  ...   
2          bronze    60.0  buy_it_now                  MLA568881256  ...   
3          silver   580.0  buy_it_now                          None  ...   
4          bronze    30.0  buy_it_now                  MLA566354576  ...   

     71    72    73    74    75    76    77    78    79    80  
0  None  None  None  None  None  None  None  None  None  None  
1  None  None  Non

In [8]:
df.head()

,warranty,condition,base_price,seller_id,site_id,listing_type_id,price,buying_mode,listing_source,parent_item_id,...,71,72,73,74,75,76,77,78,79,80
0,None,new,80.0,74952096,MLA,bronze,80.0,buy_it_now,,MLA568261029,...,None,None,None,None,None,None,None,None,None,None
1,NUESTRA REPUTACION,used,2650.0,42093335,MLA,silver,2650.0,buy_it_now,,MLA561574487,...,None,None,None,None,None,None,None,None,None,None
2,None,used,60.0,133384258,MLA,bronze,60.0,buy_it_now,,MLA568881256,...,None,None,None,None,None,None,None,None,None,None
3,None,new,580.0,143001605,MLA,silver,580.0,buy_it_now,,None,...,None,None,None,None,None,None,None,None,None,None
4,MI REPUTACION.,used,30.0,96873449,MLA,bronze,30.0,buy_it_now,,MLA566354576,...,None,None,None,None,None,None,None,None,None,None
